In [1]:
import ee
import os
import geojson
#import matplotlib.pyplot as plt
#import numpy as np
import IPython.display as disp
#from geemap import png_to_gif
#from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
import requests
import time
ee.Initialize()
%matplotlib inline

In [2]:
os.listdir("Locations/")

['algeciras.geojson',
 'ferrol.geojson',
 'gibraltar.geojson',
 'gibraltar_peq.geojson',
 'shangai.geojson',
 'shenzen.geojson',
 'shenzen_2.geojson',
 'shenzhen_bay.geojson',
 'south_sicilia.geojson']

In [3]:
filename = 'shangai.geojson'
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


start_date = "2019-01-01" # 
end_date = "2021-12-31" #

ffa_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filter(ee.Filter.eq('instrumentMode', 'IW'))
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))


im_list = ffa_db.toList(ffa_db.size())
size = ffa_db.size().getInfo()


mode = "VH"

if mode == "VV":
    min_vision = -25
    max_vision = 10
elif mode == "VH":
    min_vision = -20
    max_vision = 0

acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
dates = [time.strftime("-%Y%m%d-%H%M%S", time.gmtime(acq_time/1000)) for acq_time in acq_times]

len(dates)

213

In [7]:
ffa_db.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S1_GRD',
 'version': 1649366056909206,
 'properties': {'date_range': [1412294400000, 1649289600000],
  'period': 0,
  'type_name': 'ImageCollection',
  'keywords': ['backscattering',
   'copernicus',
   'esa',
   'eu',
   'polarization',
   'radar',
   'sar',
   'sentinel'],
  'description': '<p>The Sentinel-1 mission provides data from a dual-polarization\nC-band Synthetic Aperture Radar (SAR) instrument at 5.405GHz (C band).  This collection\nincludes the S1 Ground Range Detected (GRD) scenes, processed\nusing the Sentinel-1 Toolbox to generate a calibrated, ortho-corrected\nproduct. The collection is updated daily. New assets are ingested\nwithin two days after they become available.</p>\n<p>This collection\ncontains all of the GRD scenes.  Each scene has one of 3 resolutions\n(10, 25 or 40 meters), 4 band combinations (corresponding to\nscene polarization) and 3 instrument modes.  Use of the collection\nin a mosaic contex

In [16]:

for i,date in enumerate(dates):
    url = ee.Image(im_list.get(i)).select(mode).clip(aoi).getThumbURL({'min': min_vision, 'max': max_vision})

    img_data = requests.get(url).content
    with open(r'Database/Shenzhen bay/shenzen_' + str(i+1) + date+ '.png', 'wb') as handler:
        handler.write(img_data)

KeyboardInterrupt: 

In [21]:
url = ee.Image(im_list.get(2)).select("VV").clip(aoi).getThumbURL({'min': -25, 'max': max_vision})
disp.Image(url=url, width=1200)

1607